# pylof のテスト

<b>Local Outlier Factor algorithm</b>（LOF）のPython実装事例が下記のURLにありました。

https://github.com/damjankuznar/pylof

こちらが使えそうか確認します。

---> データ数が多すぎたせいか、３時間かけても１件目の処理すら完了しなかったため、本日はやむなく実行を強制終了しています。

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_ptna_conversation.csv',
]
copied_csv_file_paths = TestTool.copy_testdata_csv(learning_dir, csv_file_names)
copied_csv_file_paths

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]


['/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv']

## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
    
    text_array.py における TF-IDF 処理では、
    以下の通り「する」をストップワード化指定しております
    
    class TextArray:
        :
        def __build_vectorizer(self):
            :
            vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b', stop_words=['する'])
'''
#from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
from prototype.modules.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, copied_csv_file_paths, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/29 PM 08:10:13 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017/03/29 PM 08:10:13 TextArray#__init__ start
2017/03/29 PM 08:10:15 TextArray#to_vec start
2017/03/29 PM 08:10:15 TextArray#to_vec end


In [5]:
X.shape[1]

671

In [6]:
n_sample = X.shape[0]
n_feature = X.shape[1]
print("sample=%d, feature=%d" % (n_sample, n_feature))

sample=4559, feature=671


### (2-2) 外れデータのTF-IDFベクター

<b><a href="12.ipynb">こちらで再検証した時と同じ質問文</a></b>をつかっております。

In [7]:
'''
    マイオペのプロダクション・コードと同じように、
    訓練データ作成時と同じベクトライザーを使用します。
'''
#from learning.core.training_set.text_array import TextArray
from prototype.modules.text_array import TextArray

test_X = [
    '要素技術は自然languageの機械learningですか？', # まったくfeatureが抽出されない質問文

    '人生相談をしたいのですが？', # featureが１件抽出されるな質問文「する」を含む
    '難解なプログラミング技術を調達？', # featureが１件抽出される質問文「する」を含まない

    '何か習い事をしますか？', # featureが２件抽出される質問文「する」を含む
    '何か習い事がいいですか？', # featureが２件抽出される質問文「する」を含まない
    
    '何か習い事をしたほうがいいですか？', # featureが３件抽出される質問文「する」を含む
    '何か習い事を行うほうがいいですか？', # featureが３件抽出される質問文「する」を含まない
    
    '会社を辞めたいのですが誰に相談するのがいいですか？', # featureが４件抽出される質問文「する」を含む
    '会社を辞めたいのですが誰に相談を行うのがいいですか？', # featureが４件抽出される質問文「する」を含まない

    '有給休暇を取って海外に行き旅行する意向があります。', # featureが５件抽出される質問文「する」を含む
    '有給休暇を取って海外に行きたいと思っています。', # featureが５件抽出される質問文「する」を含まない
]
vectorizer = training_set.body_array.vectorizer
text_array = TextArray(test_X, vectorizer=vectorizer)

'''
    外れデータのTF-IDFベクターを取得
'''
X_error = text_array.to_vec()

2017/03/29 PM 08:10:16 TextArray#__init__ start
2017/03/29 PM 08:10:16 TextArray#to_vec start
2017/03/29 PM 08:10:16 TextArray#to_vec end


In [8]:
'''
    外れデータのボキャブラリーを確認
'''
vocabulary = text_array._vectorizer.vocabulary_
dumped_features = TestTool.get_dumped_features(X_error, vocabulary)
for d in dumped_features:
    print(d)

index=0[]
index=1[]
index=2[]
index=3[何=1.000]
index=4[いい=0.707 何=0.707]
index=5[いい=0.707 何=0.707]
index=6[いい=0.577 何=0.577 行う=0.577]
index=7[いい=0.707 誰=0.707]
index=8[いい=0.577 行う=0.577 誰=0.577]
index=9[海外=0.707 行く=0.707]
index=10[思う=0.577 海外=0.577 行く=0.577]


## (3) LOFの計算

In [9]:
'''
    データ形式を array へ変換
'''
arr_train = X.toarray()
arr_error = X_error.toarray()

In [10]:
from prototype.modules.lof import LOF
from time import time

'''
    LOF オブジェクト初期化
'''
print("Initialize LOF instance...")
t0 = time()

lof = LOF(arr_train)

print("done in %0.3fs." % (time() - t0))

'''
    LOF の計算
'''
print("Calculating local outlier factor...")
t0 = time()

for instance in arr_error:
    value = lof.local_outlier_factor(5, instance)
    print(value, instance)

print("done in %0.3fs." % (time() - t0))

Initialize LOF instance...
done in 3.726s.
Calculating local outlier factor...


KeyboardInterrupt: 

### ３時間ほど待っていたのですが、ループの１回目すら完了する様子がなかったため、実行を中止致しました。

データ数を減らすなどして、後日、再試行したいと思います。